In [ ]:
##############################
# 1. Mount Google Drive
##############################
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#######################################################
# 2. Select whether to train or load an existing model
#######################################################
TRAIN_FROM_SCRATCH = False
LOAD_CHECKPOINT_PATH = "/content/drive/MyDrive/TrainedModels/UNET/final_model_unetpp_comb_ep_45.pth"
FINAL_MODEL_SAVE_PATH = "/content/drive/MyDrive/TrainedModels/UNET/final_model_unetpp_comb_ep_45.pth"

##############################################
# 2.1 Choose which architecture to use
##############################################
# Possible values: "unet", "unet++", "unet3+"
MODEL_ARCHITECTURE = "unet++"

##############################################
# 2.2 Choose which loss function to use
##############################################
# Possible values: "CE", "Dice", "Combined"
LOSS_TYPE = "Combined"  # For example, Cross Entropy only

#####################################
# 3. Imports & Global Setup
#####################################
import os
import cv2
import json
import math
import torch
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
import matplotlib.cm as cm
import time
import datetime
import torch.nn.functional as F

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import scipy.ndimage as ndimage

# Use serif font for plots
matplotlib.rc('font', family='serif', serif=['DejaVu Serif'])
matplotlib.rc('mathtext', fontset='cm')

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

#####################################
# 4. Create root output directories
#####################################
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
ROOT_OUTPUT_DIR = f"/content/drive/MyDrive/final_model_unetpp_comb_ep_45_metrics"
os.makedirs(ROOT_OUTPUT_DIR, exist_ok=True)
print(f"Created root output directory: {ROOT_OUTPUT_DIR}")

SIGNAL_ERROR_OUTPUT_DIR = os.path.join(ROOT_OUTPUT_DIR, "SignalErrors")
DURATIONS_OUTPUT_DIR = os.path.join(ROOT_OUTPUT_DIR, "TrainingDurations")
EPOCH_METRICS_OUTPUT_DIR = os.path.join(ROOT_OUTPUT_DIR, "EpochMetrics")
FINAL_METRICS_OUTPUT_DIR = os.path.join(ROOT_OUTPUT_DIR, "FinalMetrics")
TOTAL_TRAIN_TIME_OUTPUT_DIR = os.path.join(ROOT_OUTPUT_DIR, "TotalTrainTime")

for d in [
    SIGNAL_ERROR_OUTPUT_DIR, DURATIONS_OUTPUT_DIR,
    EPOCH_METRICS_OUTPUT_DIR, FINAL_METRICS_OUTPUT_DIR,
    TOTAL_TRAIN_TIME_OUTPUT_DIR
]:
    os.makedirs(d, exist_ok=True)

#####################################
# 5. Custom Variables
#####################################
custom_colors = [
    '#000075', '#C2185B', '#FF1744', '#FF5722', '#FF9800',
    '#FFC107', '#CDDC39', '#8BC34A', '#009688', '#2979FF'
]
custom_class_names = [
    "Background", "RNFL", "GCL+IPL", "INL", "OPL",
    "ONL", "IS", "OS", "RPE", "Choroid"
]
custom_boundary_colors = [
    '#C2185B', '#FF1744', '#FF5722', '#FF9800', '#FFC107',
    '#CDDC39', '#8BC34A', '#009688', '#2979FF'
]
custom_cmap = mcolors.ListedColormap(custom_colors)

VALIDATION_CUSTOM_X_LIMITS = None
VALIDATION_CUSTOM_Y_LIMITS = None
TEST_CUSTOM_X_LIMITS = None
TEST_CUSTOM_Y_LIMITS = None
VALIDATION_OVERALL_CUSTOM_X_LIMITS = None
VALIDATION_OVERALL_CUSTOM_Y_LIMITS = None
TEST_OVERALL_CUSTOM_X_LIMITS = None
TEST_OVERALL_CUSTOM_Y_LIMITS = None

#####################################
# 6. Data Augmentations & Preprocessing
#####################################
class RandomHorizontalFlip:
    def __init__(self, p=0.5):
        self.p = p
    def __call__(self, image, mask):
        if random.random() < self.p:
            image = np.flip(image, axis=2).copy()
            mask  = np.flip(mask, axis=1).copy()
        return image, mask

class RandomElasticDeform:
    def __init__(self, alpha=34, sigma=4, p=0.5):
        self.alpha = alpha
        self.sigma = sigma
        self.p = p
    def __call__(self, image, mask):
        if random.random() < self.p:
            c, h, w = image.shape
            dx = ndimage.gaussian_filter((np.random.rand(h, w)*2 - 1),
                                         self.sigma, mode="reflect") * self.alpha
            dy = ndimage.gaussian_filter((np.random.rand(h, w)*2 - 1),
                                         self.sigma, mode="reflect") * self.alpha
            x, y = np.meshgrid(np.arange(w), np.arange(h))
            map_x = (x + dx).astype(np.float32)
            map_y = (y + dy).astype(np.float32)

            image_2d = image[0]
            distorted_image_2d = cv2.remap(image_2d, map_x, map_y,
                                           interpolation=cv2.INTER_LINEAR,
                                           borderMode=cv2.BORDER_REFLECT_101)
            distorted_mask = cv2.remap(mask.astype(np.float32), map_x, map_y,
                                       interpolation=cv2.INTER_NEAREST,
                                       borderMode=cv2.BORDER_REFLECT_101).astype(mask.dtype)

            distorted_image = np.expand_dims(distorted_image_2d, axis=0)
            return distorted_image, distorted_mask
        return image, mask

class RandomBrightnessContrast:
    def __init__(self, brightness_range=0.2, contrast_range=0.2, p=0.5):
        self.brightness_range = brightness_range
        self.contrast_range = contrast_range
        self.p = p
    def __call__(self, image, mask):
        if random.random() < self.p:
            brightness_shift = random.uniform(-self.brightness_range, self.brightness_range)
            contrast_factor = random.uniform(1 - self.contrast_range, 1 + self.contrast_range)
            image = image + brightness_shift
            image = image * contrast_factor
        return image, mask

class CLAHE_ZScore:
    def __init__(self, clipLimit=2.0, tileGridSize=(8,8)):
        self.clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    def __call__(self, image, mask):
        image_uint8 = np.clip(image[0], 0, 255).astype(np.uint8)
        clahe_img = self.clahe.apply(image_uint8)
        clahe_img = clahe_img.astype(np.float32)
        mean = np.mean(clahe_img)
        std = np.std(clahe_img)
        if std < 1e-6:
            std = 1e-6
        normalized = (clahe_img - mean) / std
        normalized = np.expand_dims(normalized, axis=0)
        return normalized, mask

class RandomSimulatedArtifacts:
    def __init__(self, p=0.5, artifact_type='occlusion'):
        self.p = p
        self.artifact_type = artifact_type
    def __call__(self, image, mask):
        if random.random() < self.p:
            c, h, w = image.shape
            if self.artifact_type == 'occlusion':
                rect_w = random.randint(w//10, w//4)
                rect_h = random.randint(h//10, h//4)
                x1 = random.randint(0, w - rect_w)
                y1 = random.randint(0, h - rect_h)
                image[:, y1:y1+rect_h, x1:x1+rect_w] = 0
            elif self.artifact_type == 'gaussian_noise':
                noise = np.random.normal(0, 0.1, size=image.shape).astype(np.float32)
                image = image + noise
        return image, mask

class ComposeAugmentations:
    def __init__(self, transforms):
        self.transforms = transforms
    def __call__(self, image, mask):
        for t in self.transforms:
            image, mask = t(image, mask)
        return image, mask

#########################################
# 7. OCTDataset Class
#########################################
class OCTDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None, target_size=(1024,256)):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.target_size = target_size
        self.image_names = sorted(os.listdir(images_dir))
        self.mask_names = sorted(os.listdir(masks_dir))
        assert len(self.image_names) == len(self.mask_names), (
            f"Mismatch: {len(self.image_names)} vs {len(self.mask_names)}"
        )

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.image_names[idx])
        msk_path = os.path.join(self.masks_dir, self.mask_names[idx])

        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
        orig_h, orig_w = image.shape[:2]

        image = cv2.resize(image, self.target_size, interpolation=cv2.INTER_AREA)
        image = np.expand_dims(image, axis=0)

        with open(msk_path, 'r') as f:
            data = json.load(f)
        boundaries = np.array(data["bds"], dtype=np.float32)
        num_boundaries = boundaries.shape[0]

        label_mask_full = np.zeros((orig_h, orig_w), dtype=np.int64)
        for col_idx in range(orig_w):
            col_bounds = np.clip(boundaries[:, col_idx], 0, orig_h-1).astype(int)
            col_bounds = np.sort(col_bounds)
            start_row = 0
            for class_id, b_row in enumerate(col_bounds):
                label_mask_full[start_row:b_row, col_idx] = class_id
                start_row = b_row
            label_mask_full[start_row:orig_h, col_idx] = num_boundaries

        label_mask = cv2.resize(label_mask_full, self.target_size, interpolation=cv2.INTER_NEAREST)
        label_mask = label_mask.astype(np.int64)

        if self.transform:
            image, label_mask = self.transform(image, label_mask)

        image_tensor = torch.from_numpy(image).float()
        label_tensor = torch.from_numpy(label_mask).long()
        return image_tensor, label_tensor

##########################################
# 8. Create DataLoaders and Transforms
##########################################
train_transform = ComposeAugmentations([
    CLAHE_ZScore(),
    RandomSimulatedArtifacts(p=0.5, artifact_type='occlusion'),
    RandomHorizontalFlip(p=0.5),
    RandomBrightnessContrast(brightness_range=0.2, contrast_range=0.2, p=0.5),
    RandomElasticDeform(alpha=34, sigma=4, p=0.5),
])

val_transform = ComposeAugmentations([CLAHE_ZScore()])
test_transform = ComposeAugmentations([CLAHE_ZScore()])

train_images_dir = "/content/drive/MyDrive/RetinaOCTData/TrainImages"
train_masks_dir = "/content/drive/MyDrive/RetinaOCTData/TrainMasks"
val_images_dir = "/content/drive/MyDrive/RetinaOCTData/ValImages"
val_masks_dir = "/content/drive/MyDrive/RetinaOCTData/ValMasks"
test_images_dir = "/content/drive/MyDrive/RetinaOCTData/TestImages"
test_masks_dir = "/content/drive/MyDrive/RetinaOCTData/TestMasks"

train_dataset = OCTDataset(train_images_dir, train_masks_dir,
                           transform=train_transform, target_size=(1024,256))
val_dataset = OCTDataset(val_images_dir, val_masks_dir,
                         transform=val_transform, target_size=(1024,256))
test_dataset = OCTDataset(test_images_dir, test_masks_dir,
                          transform=test_transform, target_size=(1024,256))

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True,
                          num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False,
                        num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False,
                         num_workers=2, pin_memory=True)

print("Total training samples:", len(train_dataset))
print("Total validation samples:", len(val_dataset))
print("Total test samples:", len(test_dataset))

#############################################
# 9. Define DoubleConv, UNet, UNet++, UNet3+
#############################################
class DoubleConv(nn.Module):
    """
    Classic UNet-style Double Convolution:
    (Conv -> BN -> ReLU) repeated twice.
    """
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    """
    A 5-level classic U-Net with DoubleConv blocks.
    """
    def __init__(self, in_channels=1, num_classes=10, base_filters=64):
        super(UNet, self).__init__()
        f = [
            base_filters,
            base_filters * 2,
            base_filters * 4,
            base_filters * 8,
            base_filters * 16,
        ]

        self.encoder1 = DoubleConv(in_channels, f[0])
        self.pool1    = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = DoubleConv(f[0], f[1])
        self.pool2    = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = DoubleConv(f[1], f[2])
        self.pool3    = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = DoubleConv(f[2], f[3])
        self.pool4    = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = DoubleConv(f[3], f[4])

        self.up4  = nn.ConvTranspose2d(f[4], f[3], kernel_size=2, stride=2)
        self.dec4 = DoubleConv(f[3]*2, f[3])
        self.up3  = nn.ConvTranspose2d(f[3], f[2], kernel_size=2, stride=2)
        self.dec3 = DoubleConv(f[2]*2, f[2])
        self.up2  = nn.ConvTranspose2d(f[2], f[1], kernel_size=2, stride=2)
        self.dec2 = DoubleConv(f[1]*2, f[1])
        self.up1  = nn.ConvTranspose2d(f[1], f[0], kernel_size=2, stride=2)
        self.dec1 = DoubleConv(f[0]*2, f[0])

        self.final_conv = nn.Conv2d(f[0], num_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.encoder1(x)
        x1p = self.pool1(x1)
        x2 = self.encoder2(x1p)
        x2p = self.pool2(x2)
        x3 = self.encoder3(x2p)
        x3p = self.pool3(x3)
        x4 = self.encoder4(x3p)
        x4p = self.pool4(x4)

        x5 = self.bottleneck(x4p)

        d4 = self.up4(x5)
        d4 = torch.cat([x4, d4], dim=1)
        d4 = self.dec4(d4)

        d3 = self.up3(d4)
        d3 = torch.cat([x3, d3], dim=1)
        d3 = self.dec3(d3)

        d2 = self.up2(d3)
        d2 = torch.cat([x2, d2], dim=1)
        d2 = self.dec2(d2)

        d1 = self.up1(d2)
        d1 = torch.cat([x1, d1], dim=1)
        d1 = self.dec1(d1)

        out = self.final_conv(d1)
        return out

class UNetPlusPlus(nn.Module):
    """
    A 5-level UNet++ with properly fixed upsampling channels.
    """
    def __init__(self, in_channels=1, num_classes=10, base_filters=64):
        super(UNetPlusPlus, self).__init__()
        self.filters = [
            base_filters,
            base_filters * 2,
            base_filters * 4,
            base_filters * 8,
            base_filters * 16,
        ]

        self.conv0_0 = DoubleConv(in_channels, self.filters[0])
        self.conv1_0 = DoubleConv(self.filters[0], self.filters[1])
        self.conv2_0 = DoubleConv(self.filters[1], self.filters[2])
        self.conv3_0 = DoubleConv(self.filters[2], self.filters[3])
        self.conv4_0 = DoubleConv(self.filters[3], self.filters[4])

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.up0_1 = nn.ConvTranspose2d(self.filters[1], self.filters[0], kernel_size=2, stride=2)
        self.up1_1 = nn.ConvTranspose2d(self.filters[2], self.filters[1], kernel_size=2, stride=2)
        self.up2_1 = nn.ConvTranspose2d(self.filters[3], self.filters[2], kernel_size=2, stride=2)
        self.up3_1 = nn.ConvTranspose2d(self.filters[4], self.filters[3], kernel_size=2, stride=2)

        self.up0_2 = nn.ConvTranspose2d(self.filters[1], self.filters[0], kernel_size=2, stride=2)
        self.up1_2 = nn.ConvTranspose2d(self.filters[2], self.filters[1], kernel_size=2, stride=2)
        self.up2_2 = nn.ConvTranspose2d(self.filters[3], self.filters[2], kernel_size=2, stride=2)

        self.up0_3 = nn.ConvTranspose2d(self.filters[1], self.filters[0], kernel_size=2, stride=2)
        self.up1_3 = nn.ConvTranspose2d(self.filters[2], self.filters[1], kernel_size=2, stride=2)

        self.up0_4 = nn.ConvTranspose2d(self.filters[1], self.filters[0], kernel_size=2, stride=2)

        self.conv0_1 = DoubleConv(self.filters[0]*2, self.filters[0])
        self.conv1_1 = DoubleConv(self.filters[1]*2, self.filters[1])
        self.conv2_1 = DoubleConv(self.filters[2]*2, self.filters[2])
        self.conv3_1 = DoubleConv(self.filters[3]*2, self.filters[3])

        self.conv0_2 = DoubleConv(self.filters[0]*3, self.filters[0])
        self.conv1_2 = DoubleConv(self.filters[1]*3, self.filters[1])
        self.conv2_2 = DoubleConv(self.filters[2]*3, self.filters[2])

        self.conv0_3 = DoubleConv(self.filters[0]*4, self.filters[0])
        self.conv1_3 = DoubleConv(self.filters[1]*4, self.filters[1])

        self.conv0_4 = DoubleConv(self.filters[0]*5, self.filters[0])

        self.final_conv = nn.Conv2d(self.filters[0], num_classes, kernel_size=1)

    def forward(self, x):
        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x2_0 = self.conv2_0(self.pool(x1_0))
        x3_0 = self.conv3_0(self.pool(x2_0))
        x4_0 = self.conv4_0(self.pool(x3_0))

        # x0_1
        x0_1_up = self.up0_1(x1_0)
        x0_1 = self.conv0_1(torch.cat([x0_0, x0_1_up], dim=1))

        # x1_1
        x1_1_up = self.up1_1(x2_0)
        x1_1 = self.conv1_1(torch.cat([x1_0, x1_1_up], dim=1))

        # x2_1
        x2_1_up = self.up2_1(x3_0)
        x2_1 = self.conv2_1(torch.cat([x2_0, x2_1_up], dim=1))

        # x3_1
        x3_1_up = self.up3_1(x4_0)
        x3_1 = self.conv3_1(torch.cat([x3_0, x3_1_up], dim=1))

        # x0_2
        x0_2_up = self.up0_2(x1_1)
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, x0_2_up], dim=1))

        # x1_2
        x1_2_up = self.up1_2(x2_1)
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, x1_2_up], dim=1))

        # x2_2
        x2_2_up = self.up2_2(x3_1)
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, x2_2_up], dim=1))

        # x0_3
        x0_3_up = self.up0_3(x1_2)
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, x0_3_up], dim=1))

        # x1_3
        x1_3_up = self.up1_3(x2_2)
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, x1_3_up], dim=1))

        # x0_4
        x0_4_up = self.up0_4(x1_3)
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, x0_4_up], dim=1))

        out = self.final_conv(x0_4)
        return out

class UNet3Plus(nn.Module):
    """
    A 5-level UNet 3+ with multi-scale feature aggregation + deep supervision,
    so we have actual parameters to train.
    """
    def __init__(self, in_channels=1, num_classes=10, base_filters=64):
        super(UNet3Plus, self).__init__()
        self.f = [
            base_filters,
            base_filters * 2,
            base_filters * 4,
            base_filters * 8,
            base_filters * 16,
        ]

        self.conv1 = DoubleConv(in_channels, self.f[0])
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = DoubleConv(self.f[0], self.f[1])
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = DoubleConv(self.f[1], self.f[2])
        self.pool3 = nn.MaxPool2d(2, 2)
        self.conv4 = DoubleConv(self.f[2], self.f[3])
        self.pool4 = nn.MaxPool2d(2, 2)
        self.conv5 = DoubleConv(self.f[3], self.f[4])

        agg_in_ch = sum(self.f)  # 64 + 128 + 256 + 512 + 1024 = 1984

        self.conv_x4d = DoubleConv(agg_in_ch, self.f[3])
        self.conv_x3d = DoubleConv(agg_in_ch, self.f[2])
        self.conv_x2d = DoubleConv(agg_in_ch, self.f[1])
        self.conv_x1d = DoubleConv(agg_in_ch, self.f[0])

        self.out_conv4 = nn.Conv2d(self.f[3], num_classes, kernel_size=1)
        self.out_conv3 = nn.Conv2d(self.f[2], num_classes, kernel_size=1)
        self.out_conv2 = nn.Conv2d(self.f[1], num_classes, kernel_size=1)
        self.out_conv1 = nn.Conv2d(self.f[0], num_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv2(self.pool1(x1))
        x3 = self.conv3(self.pool2(x2))
        x4 = self.conv4(self.pool3(x3))
        x5 = self.conv5(self.pool4(x4))

        size1 = x1.size()[2:]
        size2 = x2.size()[2:]
        size3 = x3.size()[2:]
        size4 = x4.size()[2:]

        x4_d_in = torch.cat([
            F.interpolate(x1, size=size4, mode='bilinear', align_corners=True),
            F.interpolate(x2, size=size4, mode='bilinear', align_corners=True),
            F.interpolate(x3, size=size4, mode='bilinear', align_corners=True),
            x4,
            F.interpolate(x5, size=size4, mode='bilinear', align_corners=True)
        ], dim=1)
        x4_d = self.conv_x4d(x4_d_in)

        x3_d_in = torch.cat([
            F.interpolate(x1, size=size3, mode='bilinear', align_corners=True),
            F.interpolate(x2, size=size3, mode='bilinear', align_corners=True),
            x3,
            F.interpolate(x4_d, size=size3, mode='bilinear', align_corners=True),
            F.interpolate(x5, size=size3, mode='bilinear', align_corners=True)
        ], dim=1)
        x3_d = self.conv_x3d(x3_d_in)

        x2_d_in = torch.cat([
            F.interpolate(x1, size=size2, mode='bilinear', align_corners=True),
            x2,
            F.interpolate(x3_d, size=size2, mode='bilinear', align_corners=True),
            F.interpolate(x4_d, size=size2, mode='bilinear', align_corners=True),
            F.interpolate(x5, size=size2, mode='bilinear', align_corners=True)
        ], dim=1)
        x2_d = self.conv_x2d(x2_d_in)

        x1_d_in = torch.cat([
            x1,
            F.interpolate(x2_d, size=size1, mode='bilinear', align_corners=True),
            F.interpolate(x3_d, size=size1, mode='bilinear', align_corners=True),
            F.interpolate(x4_d, size=size1, mode='bilinear', align_corners=True),
            F.interpolate(x5, size=size1, mode='bilinear', align_corners=True)
        ], dim=1)
        x1_d = self.conv_x1d(x1_d_in)

        side4 = self.out_conv4(x4_d)
        side3 = self.out_conv3(x3_d)
        side2 = self.out_conv2(x2_d)
        side1 = self.out_conv1(x1_d)

        side4_up = F.interpolate(side4, size=size1, mode='bilinear', align_corners=True)
        side3_up = F.interpolate(side3, size=size1, mode='bilinear', align_corners=True)
        side2_up = F.interpolate(side2, size=size1, mode='bilinear', align_corners=True)

        final = (side1 + side2_up + side3_up + side4_up) / 4.0
        return final

#############################################
# 10. Metric Functions and Loss Definitions
#############################################
ce_loss = nn.CrossEntropyLoss()

class MultiClassDiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(MultiClassDiceLoss, self).__init__()
        self.smooth = smooth
    def forward(self, logits, targets):
        probs = nn.functional.softmax(logits, dim=1)
        num_classes = logits.shape[1]
        targets_one_hot = nn.functional.one_hot(targets, num_classes=num_classes)
        targets_one_hot = targets_one_hot.permute(0, 3, 1, 2).float()
        intersection = torch.sum(probs * targets_one_hot, dim=(0,2,3))
        union = torch.sum(probs, dim=(0,2,3)) + torch.sum(targets_one_hot, dim=(0,2,3))
        dice_per_class = (2.0 * intersection + self.smooth) / (union + self.smooth)
        dice_score = torch.mean(dice_per_class)
        dice_loss_val = 1.0 - dice_score
        return dice_loss_val

dice_loss = MultiClassDiceLoss(smooth=1.0)

def combined_loss(logits, targets, alpha=0.5):
    loss_ce = ce_loss(logits, targets)
    loss_d = dice_loss(logits, targets)
    return alpha * loss_ce + (1.0 - alpha) * loss_d

def create_confusion_matrix(num_classes):
    return np.zeros((num_classes, num_classes), dtype=np.float64)

def update_confusion_matrix(pred, target, conf_mat):
    pred_np = pred.ravel()
    target_np = target.ravel()
    for t, p in zip(target_np, pred_np):
        if 0 <= t < conf_mat.shape[0] and 0 <= p < conf_mat.shape[0]:
            conf_mat[t, p] += 1

def compute_classwise_iou_dice(conf_mat):
    C = conf_mat.shape[0]
    ious, dices = [], []
    for c in range(C):
        tp = conf_mat[c, c]
        fp = conf_mat[:, c].sum() - tp
        fn = conf_mat[c, :].sum() - tp
        denom_iou = tp + fp + fn
        iou = tp / denom_iou if denom_iou > 1e-10 else 0
        ious.append(iou)
        denom_dice = (2 * tp) + fp + fn
        dice = (2 * tp) / denom_dice if denom_dice > 1e-10 else 0
        dices.append(dice)
    mean_iou = float(np.mean(ious))
    mean_dice = float(np.mean(dices))
    return ious, dices, mean_iou, mean_dice

def compute_precision_recall_f1(conf_mat):
    C = conf_mat.shape[0]
    precisions, recalls, f1s = [], [], []
    for c in range(C):
        tp = conf_mat[c, c]
        fp = conf_mat[:, c].sum() - tp
        fn = conf_mat[c, :].sum() - tp
        prec = tp / (tp + fp + 1e-15)
        rec = tp / (tp + fn + 1e-15)
        f1 = 2 * prec * rec / (prec + rec + 1e-15)
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
    macro_prec = float(np.mean(precisions))
    macro_rec = float(np.mean(recalls))
    macro_f1 = float(np.mean(f1s))
    return (precisions, recalls, f1s, macro_prec, macro_rec, macro_f1)

def extract_boundaries(label_map, num_boundaries=9):
    H, W = label_map.shape
    boundaries = np.full((num_boundaries, W), H-1, dtype=np.float32)
    for c in range(W):
        col_vals = label_map[:, c]
        for i in range(num_boundaries):
            indices = np.where((col_vals[:-1] == i) & (col_vals[1:] == i+1))[0]
            if indices.size > 0:
                boundaries[i, c] = indices[0] + 1
    return boundaries

def single_sample_metrics(pred_np, target_np, num_classes=10):
    correct_pixels = np.sum(pred_np == target_np)
    total_pixels = target_np.size
    sample_pix_acc = correct_pixels / total_pixels
    sample_conf = np.zeros((num_classes, num_classes), dtype=np.float64)
    for t, p in zip(target_np.ravel(), pred_np.ravel()):
        if 0 <= t < num_classes and 0 <= p < num_classes:
            sample_conf[t, p] += 1
    ious, _, mean_iou, _ = compute_classwise_iou_dice(sample_conf)
    return sample_pix_acc, mean_iou

#############################################
# 11. Train & Validate Loop
#############################################
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for images, masks in loader:
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

def validate_epoch(model, loader, criterion, num_classes=10, num_boundaries=9):
    model.eval()
    running_loss = 0.0
    total_correct = 0
    total_pixels = 0
    conf_mat = create_confusion_matrix(num_classes)
    sample_rmse_list = []
    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            running_loss += loss.item()
            preds = torch.argmax(outputs, dim=1)
            total_correct += (preds == masks).sum().item()
            total_pixels += masks.numel()

            for i in range(preds.shape[0]):
                p_np = preds[i].cpu().numpy()
                t_np = masks[i].cpu().numpy()
                update_confusion_matrix(p_np, t_np, conf_mat)
                gt_bds = extract_boundaries(t_np, num_boundaries)
                pred_bds = extract_boundaries(p_np, num_boundaries)
                H = t_np.shape[0]
                bd_diff = (pred_bds - gt_bds) / float(H)
                rmse_vals = np.sqrt(np.mean(bd_diff**2, axis=1))
                sample_rmse_list.append(rmse_vals)

    if len(sample_rmse_list) > 0:
        boundary_rmse = np.mean(sample_rmse_list, axis=0)
    else:
        boundary_rmse = np.zeros(num_boundaries, dtype=np.float32)

    val_loss = running_loss / len(loader)
    pixel_acc = total_correct / total_pixels

    ious, dices, mean_iou, mean_dice = compute_classwise_iou_dice(conf_mat)
    (precs, recs, f1s,
     macro_prec, macro_rec, macro_f1) = compute_precision_recall_f1(conf_mat)

    return (val_loss, pixel_acc,
            ious, dices, mean_iou, mean_dice,
            precs, recs, f1s, macro_prec, macro_rec, macro_f1,
            boundary_rmse, conf_mat)

##########################################
# 12. Create model, optimizer, and pick the loss
##########################################
print(f"Selected architecture: {MODEL_ARCHITECTURE}")
print(f"Selected loss function: {LOSS_TYPE}")

if MODEL_ARCHITECTURE.lower() == "unet":
    model = UNet(in_channels=1, num_classes=10).to(device)
elif MODEL_ARCHITECTURE.lower() == "unet++":
    model = UNetPlusPlus(in_channels=1, num_classes=10).to(device)
elif MODEL_ARCHITECTURE.lower() == "unet3+":
    model = UNet3Plus(in_channels=1, num_classes=10).to(device)
else:
    raise ValueError(f"Unsupported model architecture: {MODEL_ARCHITECTURE}")

if LOSS_TYPE == "CE":
    criterion = ce_loss
elif LOSS_TYPE == "Dice":
    criterion = dice_loss
elif LOSS_TYPE == "Combined":
    criterion = lambda logits, targets: combined_loss(logits, targets, alpha=0.5)
else:
    raise ValueError(f"Unsupported LOSS_TYPE: {LOSS_TYPE}")

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=45, eta_min=1e-5)
num_epochs = 45
best_iou = 0.0

##########################################
# 13. Decide whether to train or load
##########################################
if TRAIN_FROM_SCRATCH:
    print("\n=== Option set to TRAIN_FROM_SCRATCH = True. Training from scratch. ===")
else:
    if (LOAD_CHECKPOINT_PATH is not None) and (os.path.isfile(LOAD_CHECKPOINT_PATH)):
        print(f"\n=== Loading model from checkpoint: {LOAD_CHECKPOINT_PATH} ===")
        model.load_state_dict(torch.load(LOAD_CHECKPOINT_PATH, map_location=device))
        model.eval()
        print("Model loaded and set to evaluation mode. Skipping training.")
    else:
        print("TRAIN_FROM_SCRATCH = False but checkpoint not found. Training from scratch.")
        TRAIN_FROM_SCRATCH = True

##########################################
# 14. Training (if requested)
##########################################
epoch_durations = []
all_epoch_metrics = []

if TRAIN_FROM_SCRATCH:
    print("\n=== Training Model ===")
    total_train_start_time = time.time()

    for epoch in range(num_epochs):
        epoch_start_time = time.time()

        train_loss = train_epoch(model, train_loader, optimizer, criterion)
        scheduler.step()

        (val_loss, val_pix_acc,
         val_ious, val_dices, val_mean_iou, val_mean_dice,
         val_precs, val_recs, val_f1s,
         val_macro_prec, val_macro_rec, val_macro_f1,
         val_boundary_rmse, _) = validate_epoch(model, val_loader, criterion)

        epoch_end_time = time.time()
        epoch_duration = epoch_end_time - epoch_start_time
        epoch_durations.append(epoch_duration)

        epoch_metrics = {
            "epoch": epoch+1,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "val_pix_acc": val_pix_acc,
            "val_mean_iou": val_mean_iou,
            "val_mean_dice": val_mean_dice,
            "val_macro_prec": val_macro_prec,
            "val_macro_rec": val_macro_rec,
            "val_macro_f1": val_macro_f1
        }
        all_epoch_metrics.append(epoch_metrics)

        print(f"\nEpoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss:          {train_loss:.4f}")
        print(f"  Val Loss:            {val_loss:.4f}")
        print(f"  Val Pixel Accuracy:  {val_pix_acc:.4f}")
        print(f"  Val Mean IoU:        {val_mean_iou:.4f}")
        print(f"  Val Mean Dice:       {val_mean_dice:.4f}")
        print(f"  Val Macro Precision: {val_macro_prec:.4f}")
        print(f"  Val Macro Recall:    {val_macro_rec:.4f}")
        print(f"  Val Macro F1:        {val_macro_f1:.4f}")

        print("  -- Classwise Metrics --")
        for c_idx in range(len(val_ious)):
            print(f"     Class {c_idx}: IoU={val_ious[c_idx]:.4f}, "
                  f"Dice={val_dices[c_idx]:.4f}, "
                  f"Prec={val_precs[c_idx]:.4f}, "
                  f"Rec={val_recs[c_idx]:.4f}, "
                  f"F1={val_f1s[c_idx]:.4f}")

        print("  -- Per-Boundary RMSE (Val) --")
        for b_idx, rmse_val in enumerate(val_boundary_rmse):
            print(f"     Boundary {b_idx}: RMSE={rmse_val:.4f}")

        print(f"  => Epoch {epoch+1} took {epoch_duration:.2f} seconds.")

        if val_mean_iou > best_iou:
            best_iou = val_mean_iou
            torch.save(model.state_dict(), f"best_model_epoch_{epoch+1}.pth")
            print(f"  => New best model saved with Mean IoU: {best_iou:.4f}")

    total_train_end_time = time.time()
    total_training_time = total_train_end_time - total_train_start_time
    print(f"\nTraining complete. Total training time: {total_training_time:.2f} seconds.")

    total_time_txt_path = os.path.join(TOTAL_TRAIN_TIME_OUTPUT_DIR, "total_training_time.txt")
    with open(total_time_txt_path, "w") as f:
        f.write(f"Total training time (seconds): {total_training_time:.2f}\n")
    print(f"Total training time saved to: {total_time_txt_path}")

    for i, dur in enumerate(epoch_durations):
        print(f"Epoch {i+1} duration: {dur:.2f} seconds")

    durations_txt_path = os.path.join(DURATIONS_OUTPUT_DIR, "epoch_durations.txt")
    with open(durations_txt_path, "w") as f:
        f.write("Epoch\tDuration_seconds\n")
        for i, dur in enumerate(epoch_durations, start=1):
            f.write(f"{i}\t{dur:.6f}\n")
    print(f"Epoch durations saved to: {durations_txt_path}")

    epoch_metrics_txt_path = os.path.join(EPOCH_METRICS_OUTPUT_DIR, "epoch_metrics.txt")
    with open(epoch_metrics_txt_path, "w") as f:
        f.write("Epoch\tTrain_Loss\tVal_Loss\tVal_Pixel_Acc\tVal_Mean_IoU\tVal_Mean_Dice\tVal_Macro_Prec\tVal_Macro_Rec\tVal_Macro_F1\n")
        for em in all_epoch_metrics:
            f.write(f"{em['epoch']}\t"
                    f"{em['train_loss']:.4f}\t"
                    f"{em['val_loss']:.4f}\t"
                    f"{em['val_pix_acc']:.4f}\t"
                    f"{em['val_mean_iou']:.4f}\t"
                    f"{em['val_mean_dice']:.4f}\t"
                    f"{em['val_macro_prec']:.4f}\t"
                    f"{em['val_macro_rec']:.4f}\t"
                    f"{em['val_macro_f1']:.4f}\n")
    print(f"Epoch metrics saved to: {epoch_metrics_txt_path}")

    if len(epoch_durations) > 0:
        plt.figure()
        plt.plot(range(1, len(epoch_durations) + 1), epoch_durations, label='Epoch Duration (s)')
        plt.xlabel("Epoch")
        plt.ylabel("Duration (s)")
        plt.title("Training Time per Epoch")
        plt.legend()
        plt.show()

    torch.save(model.state_dict(), FINAL_MODEL_SAVE_PATH)
    print(f"Final model saved to: {FINAL_MODEL_SAVE_PATH}")

##########################################
# 15. Validation Metrics & Standard Deviations
##########################################
print("\n=== Validation Metrics & Standard Deviations ===")
(val_loss, val_pix_acc,
 val_ious, val_dices, val_mean_iou, val_mean_dice,
 val_precs, val_recs, val_f1s, val_macro_prec, val_macro_rec, val_macro_f1,
 val_boundary_rmse, _) = validate_epoch(model, val_loader, criterion)

print(f"Val Loss:            {val_loss:.4f}")
print(f"Val Pixel Accuracy:  {val_pix_acc:.4f}")
print(f"Val Mean IoU:        {val_mean_iou:.4f}")
print(f"Val Mean Dice:       {val_mean_dice:.4f}")
print(f"Val Macro Precision: {val_macro_prec:.4f}")
print(f"Val Macro Recall:    {val_macro_rec:.4f}")
print(f"Val Macro F1:        {val_macro_f1:.4f}")
for c_idx in range(len(val_ious)):
    print(f"  Class {c_idx}: IoU={val_ious[c_idx]:.4f}, Dice={val_dices[c_idx]:.4f}, "
          f"Prec={val_precs[c_idx]:.4f}, Rec={val_recs[c_idx]:.4f}, F1={val_f1s[c_idx]:.4f}")
print("\n-- Per-Boundary RMSE (Val) --")
for b_idx, rmse_val in enumerate(val_boundary_rmse):
    print(f"  Boundary {b_idx}: RMSE={rmse_val:.4f}")

final_val_metrics_txt = os.path.join(FINAL_METRICS_OUTPUT_DIR, "final_validation_metrics.txt")
with open(final_val_metrics_txt, "w") as f:
    f.write("Final Validation Metrics\n")
    f.write(f"Val_Loss: {val_loss:.4f}\n")
    f.write(f"Val_Pixel_Acc: {val_pix_acc:.4f}\n")
    f.write(f"Val_Mean_IoU: {val_mean_iou:.4f}\n")
    f.write(f"Val_Mean_Dice: {val_mean_dice:.4f}\n")
    f.write(f"Val_Macro_Precision: {val_macro_prec:.4f}\n")
    f.write(f"Val_Macro_Recall: {val_macro_rec:.4f}\n")
    f.write(f"Val_Macro_F1: {val_macro_f1:.4f}\n")
    f.write("\nClasswise IoU / Dice / Precision / Recall / F1:\n")
    for c_idx in range(len(val_ious)):
        f.write(
            f"Class {c_idx}\t"
            f"IoU={val_ious[c_idx]:.4f}\t"
            f"Dice={val_dices[c_idx]:.4f}\t"
            f"Prec={val_precs[c_idx]:.4f}\t"
            f"Rec={val_recs[c_idx]:.4f}\t"
            f"F1={val_f1s[c_idx]:.4f}\n"
        )
    f.write("\nPer-Boundary RMSE:\n")
    for b_idx, rmse_val in enumerate(val_boundary_rmse):
        f.write(f"Boundary {b_idx}: {rmse_val:.4f}\n")

print(f"Final validation metrics saved to: {final_val_metrics_txt}")

def compute_std_for_dataset(model, loader, device, num_classes=10):
    model.eval()
    stds_per_class = {c: [] for c in range(num_classes)}
    overall_stds = []
    with torch.no_grad():
        for images, _ in loader:
            images = images.to(device)
            outputs = model(images)
            prob_maps = torch.softmax(outputs, dim=1)
            prob_maps_np = prob_maps.cpu().numpy()
            B = prob_maps_np.shape[0]
            for i in range(B):
                sample_prob = prob_maps_np[i]
                overall_stds.append(np.std(sample_prob))
                for c in range(num_classes):
                    stds_per_class[c].append(np.std(sample_prob[c]))
    mean_stds_per_class = {c: np.mean(stds_per_class[c]) for c in range(num_classes)}
    mean_overall_std = np.mean(overall_stds)
    return mean_stds_per_class, mean_overall_std

val_std_per_class, val_overall_std = compute_std_for_dataset(model, val_loader, device)
print("\n=== Validation Set Standard Deviations ===")
for c in range(10):
    print(f"  Class {c}: Std = {val_std_per_class[c]:.4f}")
print(f"  Overall (per sample) Std: {val_overall_std:.4f}")

#############################################
# 16. Validation set visualization
#############################################
def visualize_on_batch_val(loader, model, num_boundaries=9):
    model.eval()
    batch_iter = iter(loader)
    images, masks = next(batch_iter)
    images, masks = images.to(device), masks.to(device)
    with torch.no_grad():
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        prob_maps = torch.softmax(outputs, dim=1).cpu().numpy()

    idx = 0
    img_np = images[idx].cpu().numpy()[0]
    gt_np = masks[idx].cpu().numpy()
    pred_np = preds[idx].cpu().numpy()
    prob_np = prob_maps[idx]

    H, W = img_np.shape
    mismatch = (pred_np != gt_np)
    prob_err_raw = np.zeros_like(gt_np, dtype=np.float32)
    rows, cols = np.where(mismatch)
    if rows.size > 0:
        pred_classes = pred_np[mismatch]
        prob_err_raw[mismatch] = prob_np[pred_classes, rows, cols]

    gt_bds = extract_boundaries(gt_np, num_boundaries)
    raw_bds = extract_boundaries(pred_np, num_boundaries)

    def show_image(ax, data_2d, title, cmap=None, vmin=None, vmax=None):
        flipped = np.flipud(data_2d)
        ax.set_title(title)
        ax.imshow(flipped, extent=[0, W, 0, H], origin='lower',
                  cmap=cmap, vmin=vmin, vmax=vmax)
        ax.set_xlim(0, W)
        ax.set_ylim(0, H)
        ax.set_xlabel("X (px)")
        ax.set_ylabel("Y (px)")

    fig = plt.figure(figsize=(12,32))

    ax1 = fig.add_subplot(6,1,1)
    show_image(ax1, img_np, "1) Validation Input (OCT)", cmap="gray")

    ax2 = fig.add_subplot(6,1,2)
    show_image(ax2, gt_np, "2) GT Mask", cmap=custom_cmap)
    gt_patches = [
        Patch(facecolor=custom_colors[i], edgecolor='black', label=custom_class_names[i])
        for i in range(len(custom_colors))
    ]
    ax2.legend(handles=gt_patches, loc="center left", bbox_to_anchor=(1, 0.5))

    ax3 = fig.add_subplot(6,1,3)
    show_image(ax3, img_np, "3) GT Boundaries", cmap="gray")
    for b_idx in range(gt_bds.shape[0]):
        boundary_ys = H - gt_bds[b_idx]
        ax3.plot(np.arange(W), boundary_ys,
                 color=custom_boundary_colors[b_idx % len(custom_boundary_colors)],
                 linestyle='-', linewidth=1.25, alpha=0.7)

    ax4 = fig.add_subplot(6,1,4)
    show_image(ax4, pred_np, "4) Raw Prediction", cmap=custom_cmap)

    ax5 = fig.add_subplot(6,1,5)
    show_image(ax5, img_np, "5) Raw Predicted Boundaries", cmap="gray")
    for b_idx in range(raw_bds.shape[0]):
        boundary_ys = H - raw_bds[b_idx]
        ax5.plot(np.arange(W), boundary_ys,
                 color=custom_boundary_colors[b_idx % len(custom_boundary_colors)],
                 linestyle='--', linewidth=1.875, alpha=0.8)

    ax6 = fig.add_subplot(6,1,6)
    show_image(ax6, prob_err_raw, "6) Raw Prediction Probability Heatmap",
               cmap='jet', vmin=0, vmax=1)
    plt.colorbar(ax6.images[0], ax=ax6, label="Probability Error",
                 fraction=0.046, pad=0.02, shrink=0.375)

    plt.tight_layout()
    plt.show()

print("\n--- Validation Set Visualization ---")
if len(val_loader) > 0:
    visualize_on_batch_val(val_loader, model, num_boundaries=9)

##########################################
# 17. Compute Boundary Difference for Validation
##########################################
def compute_boundary_diff_across_dataset(model, loader, device, num_boundaries=9):
    boundary_diffs_raw = [[] for _ in range(num_boundaries)]
    model.eval()
    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            for i in range(preds.shape[0]):
                pred_np = preds[i].cpu().numpy()
                gt_np = masks[i].cpu().numpy()
                gt_bds = extract_boundaries(gt_np, num_boundaries)
                pred_bds = extract_boundaries(pred_np, num_boundaries)
                H = gt_np.shape[0]
                raw_diff = (pred_bds - gt_bds) / float(H) * 100.0
                for b_idx in range(num_boundaries):
                    boundary_diffs_raw[b_idx].append(raw_diff[b_idx, :])
    return boundary_diffs_raw

def compute_overall_mean_signal(boundary_diffs):
    if not boundary_diffs or not boundary_diffs[0]:
        return None
    import numpy as np
    num_boundaries = len(boundary_diffs)
    num_samples = len(boundary_diffs[0])
    sample_mean_signals = []
    for i in range(num_samples):
        stacked = np.stack([boundary_diffs[b][i] for b in range(num_boundaries)], axis=0)
        mean_across_boundaries = np.mean(stacked, axis=0)
        sample_mean_signals.append(mean_across_boundaries)
    sample_mean_signals = np.stack(sample_mean_signals, axis=0)
    overall_mean = np.mean(sample_mean_signals, axis=0)
    return overall_mean

def plot_overall_boundary_diff_signal(overall_mean, title, custom_x_limits=None, custom_y_limits=None,
                                      output_dir=SIGNAL_ERROR_OUTPUT_DIR):
    import numpy as np
    if overall_mean is None:
        print("No data to plot.")
        return
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    x = np.arange(overall_mean.shape[0])
    fig, ax = plt.subplots(figsize=(9.6, 4.8))
    ax.plot(x, overall_mean, label="Overall Mean Boundary Error", alpha=0.9)
    ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.8)
    ax.set_title(title)
    ax.set_xlabel("Column Index (px)")
    ax.set_ylabel("Error (%)")
    if custom_x_limits is not None:
        ax.set_xlim(*custom_x_limits)
    else:
        ax.set_xlim(0, overall_mean.shape[0])
    if custom_y_limits is not None:
        ax.set_ylim(*custom_y_limits)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()

    data = np.vstack((x, overall_mean)).T
    filename = os.path.join(output_dir, title.replace(" ", "_") + ".txt")
    np.savetxt(filename, data, fmt="%.6f", header="Column_Index Overall_Mean_Error (%)")
    print(f"Data saved to {filename}")

def plot_boundary_diff_signals(boundary_diffs, title_prefix="Boundary Error Signals",
                               custom_x_limits=None, custom_y_limits=None,
                               output_dir=SIGNAL_ERROR_OUTPUT_DIR):
    import numpy as np
    num_boundaries = len(boundary_diffs)
    if not boundary_diffs[0]:
        print("No data to plot.")
        return
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    sample_width = boundary_diffs[0][0].shape[0]
    x = np.arange(sample_width)
    fig, ax = plt.subplots(figsize=(9.6, 4.8))
    for b in range(num_boundaries):
        error_signals = np.stack(boundary_diffs[b], axis=0)
        mean_error = np.mean(error_signals, axis=0)
        ax.plot(x, mean_error,
                label=f"Boundary {b}",
                color=custom_boundary_colors[b % len(custom_boundary_colors)],
                alpha=0.9)
        data = np.vstack((x, mean_error)).T
        filename = os.path.join(output_dir,
                                f"{title_prefix.replace(' ', '_')}_Boundary_{b}.txt")
        np.savetxt(filename, data, fmt="%.6f", header="Column_Index Error (%)")
        print(f"Data for Boundary {b} saved to {filename}")
    ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.8)
    ax.set_title(title_prefix)
    ax.set_xlabel("Column Index (px)")
    ax.set_ylabel("Error (%)")
    if custom_x_limits is not None:
        ax.set_xlim(*custom_x_limits)
    else:
        ax.set_xlim(0, sample_width)
    if custom_y_limits is not None:
        ax.set_ylim(*custom_y_limits)
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout()
    plt.show()

print("\n--- Validation Set Boundary Error Signal ---")
if len(val_loader) > 0:
    val_boundary_diffs_raw = compute_boundary_diff_across_dataset(model, val_loader, device, num_boundaries=9)
    plot_boundary_diff_signals(val_boundary_diffs_raw,
                               title_prefix="Validation Set Boundary Error Signal (Raw)",
                               custom_x_limits=VALIDATION_CUSTOM_X_LIMITS,
                               custom_y_limits=VALIDATION_CUSTOM_Y_LIMITS,
                               output_dir=SIGNAL_ERROR_OUTPUT_DIR)

    val_overall_raw = compute_overall_mean_signal(val_boundary_diffs_raw)
    plot_overall_boundary_diff_signal(
        val_overall_raw,
        "Validation Set Overall Boundary Error Signal (Raw)",
        custom_x_limits=VALIDATION_OVERALL_CUSTOM_X_LIMITS,
        custom_y_limits=VALIDATION_OVERALL_CUSTOM_Y_LIMITS,
        output_dir=SIGNAL_ERROR_OUTPUT_DIR
    )

##########################################
# 18. Test Set Evaluation & Visualization
##########################################
test_results = validate_epoch(model, test_loader, criterion, num_classes=10, num_boundaries=9)
(test_loss, test_pix_acc,
 test_ious, test_dices, test_mean_iou, test_mean_dice,
 test_precs, test_recs, test_f1s,
 test_mprec, test_mrec, test_mf1,
 test_bd_rmse, test_conf_mat) = test_results

print("\n=== Test Set Evaluation ===")
print(f"Test Loss:           {test_loss:.4f}")
print(f"Pixel Accuracy:      {test_pix_acc:.4f}")
print(f"Mean IoU:            {test_mean_iou:.4f}")
print(f"Mean Dice:           {test_mean_dice:.4f}")
print(f"Macro Precision:     {test_mprec:.4f}")
print(f"Macro Recall:        {test_mrec:.4f}")
print(f"Macro F1:            {test_mf1:.4f}")

for c_idx in range(len(test_ious)):
    print(f"  Class {c_idx}: IoU={test_ious[c_idx]:.4f}, Dice={test_dices[c_idx]:.4f}, "
          f"Prec={test_precs[c_idx]:.4f}, Rec={test_recs[c_idx]:.4f}, F1={test_f1s[c_idx]:.4f}")

print("\n-- Per-Boundary RMSE (Test Set) --")
for b_idx, rmse_val in enumerate(test_bd_rmse):
    print(f"  Boundary {b_idx}: RMSE={rmse_val:.4f}")

test_std_per_class, test_overall_std = compute_std_for_dataset(model, test_loader, device)
print("\n=== Test Set Standard Deviations ===")
for c in range(10):
    print(f"  Class {c}: Std = {test_std_per_class[c]:.4f}")
print(f"  Overall (per sample) Std: {test_overall_std:.4f}")

test_metrics_txt = os.path.join(FINAL_METRICS_OUTPUT_DIR, "final_test_metrics.txt")
with open(test_metrics_txt, "w") as f:
    f.write("Final Test Metrics\n")
    f.write(f"Test_Loss: {test_loss:.4f}\n")
    f.write(f"Test_Pixel_Acc: {test_pix_acc:.4f}\n")
    f.write(f"Test_Mean_IoU: {test_mean_iou:.4f}\n")
    f.write(f"Test_Mean_Dice: {test_mean_dice:.4f}\n")
    f.write(f"Test_Macro_Precision: {test_mprec:.4f}\n")
    f.write(f"Test_Macro_Recall: {test_mrec:.4f}\n")
    f.write(f"Test_Macro_F1: {test_mf1:.4f}\n")
    f.write("\nClasswise IoU / Dice / Precision / Recall / F1:\n")
    for c_idx in range(len(test_ious)):
        f.write(
            f"Class {c_idx}\t"
            f"IoU={test_ious[c_idx]:.4f}\t"
            f"Dice={test_dices[c_idx]:.4f}\t"
            f"Prec={test_precs[c_idx]:.4f}\t"
            f"Rec={test_recs[c_idx]:.4f}\t"
            f"F1={test_f1s[c_idx]:.4f}\n"
        )
    f.write("\nPer-Boundary RMSE:\n")
    for b_idx, rmse_val in enumerate(test_bd_rmse):
        f.write(f"Boundary {b_idx}: {rmse_val:.4f}\n")

print(f"Final test metrics saved to: {test_metrics_txt}")

##########################################
# 18.1 Single-Batch Visualization for the Test Set
##########################################
def visualize_on_batch_test(loader, model, num_boundaries=9):
    model.eval()
    batch_iter = iter(loader)
    images, masks = next(batch_iter)
    images, masks = images.to(device), masks.to(device)
    with torch.no_grad():
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        prob_maps = torch.softmax(outputs, dim=1).cpu().numpy()

    idx = 0
    img_np = images[idx].cpu().numpy()[0]
    gt_np = masks[idx].cpu().numpy()
    pred_np = preds[idx].cpu().numpy()
    prob_np = prob_maps[idx]

    H, W = img_np.shape
    mismatch = (pred_np != gt_np)
    prob_err_raw = np.zeros_like(gt_np, dtype=np.float32)
    rows, cols = np.where(mismatch)
    if rows.size > 0:
        pred_classes = pred_np[mismatch]
        prob_err_raw[mismatch] = prob_np[pred_classes, rows, cols]

    gt_bds = extract_boundaries(gt_np, num_boundaries)
    raw_bds = extract_boundaries(pred_np, num_boundaries)

    def show_image(ax, data_2d, title, cmap=None, vmin=None, vmax=None):
        flipped = np.flipud(data_2d)
        ax.set_title(title)
        ax.imshow(flipped, extent=[0, W, 0, H], origin='lower',
                  cmap=cmap, vmin=vmin, vmax=vmax)
        ax.set_xlim(0, W)
        ax.set_ylim(0, H)
        ax.set_xlabel("X (px)")
        ax.set_ylabel("Y (px)")

    fig = plt.figure(figsize=(12,32))

    ax1 = fig.add_subplot(6,1,1)
    show_image(ax1, img_np, "1) Test Input (OCT)", cmap="gray")

    ax2 = fig.add_subplot(6,1,2)
    show_image(ax2, gt_np, "2) GT Mask", cmap=custom_cmap)
    gt_patches = [
        Patch(facecolor=custom_colors[i], edgecolor='black', label=custom_class_names[i])
        for i in range(len(custom_colors))
    ]
    ax2.legend(handles=gt_patches, loc="center left", bbox_to_anchor=(1, 0.5))

    ax3 = fig.add_subplot(6,1,3)
    show_image(ax3, img_np, "3) GT Boundaries", cmap="gray")
    for b_idx in range(gt_bds.shape[0]):
        boundary_ys = H - gt_bds[b_idx]
        ax3.plot(np.arange(W), boundary_ys,
                 color=custom_boundary_colors[b_idx % len(custom_boundary_colors)],
                 linestyle='-', linewidth=1.25, alpha=0.7)

    ax4 = fig.add_subplot(6,1,4)
    show_image(ax4, pred_np, "4) Raw Prediction", cmap=custom_cmap)

    ax5 = fig.add_subplot(6,1,5)
    show_image(ax5, img_np, "5) Raw Predicted Boundaries", cmap="gray")
    for b_idx in range(raw_bds.shape[0]):
        boundary_ys = H - raw_bds[b_idx]
        ax5.plot(np.arange(W), boundary_ys,
                 color=custom_boundary_colors[b_idx % len(custom_boundary_colors)],
                 linestyle='--', linewidth=1.875, alpha=0.8)

    ax6 = fig.add_subplot(6,1,6)
    show_image(ax6, prob_err_raw, "6) Raw Prediction Probability Heatmap",
               cmap='jet', vmin=0, vmax=1)
    plt.colorbar(ax6.images[0], ax=ax6, label="Probability Error",
                 fraction=0.046, pad=0.02, shrink=0.375)

    plt.tight_layout()
    plt.show()

print("\n=== Single-Batch Visualization on the Test Set ===")
if len(test_loader) > 0:
    visualize_on_batch_test(test_loader, model, num_boundaries=9)

##########################################
# 18.2 Plotting Test Set Boundary Errors
##########################################
test_boundary_diffs_raw = compute_boundary_diff_across_dataset(model, test_loader, device, num_boundaries=9)

plot_boundary_diff_signals(test_boundary_diffs_raw,
                           title_prefix="Test Set Boundary Error Signal (Raw)",
                           custom_x_limits=TEST_CUSTOM_X_LIMITS,
                           custom_y_limits=TEST_CUSTOM_Y_LIMITS,
                           output_dir=SIGNAL_ERROR_OUTPUT_DIR)

test_overall_raw = compute_overall_mean_signal(test_boundary_diffs_raw)
plot_overall_boundary_diff_signal(
    test_overall_raw,
    "Test Set Overall Boundary Error Signal (Raw)",
    custom_x_limits=TEST_OVERALL_CUSTOM_X_LIMITS,
    custom_y_limits=TEST_OVERALL_CUSTOM_Y_LIMITS,
    output_dir=SIGNAL_ERROR_OUTPUT_DIR
)

print("\nDone.")